In [1]:
# Import required modules
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

# Data Viz
import matplotlib.pyplot as plt
import seaborn as sns
import hvplot.pandas

# Machine Learning
from sklearn.cluster import KMeans, AgglomerativeClustering, Birch
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

# Suppress Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [3]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [4]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [7]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
cols = ['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d',
       'price_change_percentage_60d', 'price_change_percentage_200d',
       'price_change_percentage_1y']

# Initalize and Fit
ss = StandardScaler()
ss.fit(df_market_data.loc[:, cols])


# Copy the crypto names from the original DataFrame
# Set the coin_id column as index
df2 = pd.DataFrame(ss.transform(df_market_data.loc[:, cols]), columns=cols, index=df_market_data.index)


# Display the scaled DataFrame
df2

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317
binancecoin,0.891871,1.327295,0.800214,-0.057148,0.778653,-0.188232,-0.225533
chainlink,0.011397,2.572251,1.101647,-0.490495,-0.931954,0.387759,-0.018284
cardano,0.102530,1.508001,0.648885,0.328959,-0.486349,0.065080,-0.155428
litecoin,0.077497,0.334297,0.858520,-0.012646,-0.366477,-0.486266,-0.292351


---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [10]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,12))
k

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [11]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    # 2. Fit the model to the data using `df_market_data_scaled`
    k_model.fit(df2)
    # 3. Append the model.inertia_ to the inertia list
    inertia.append(k_model.inertia_)

In [12]:
# Create a dictionary with the data to plot the Elbow curve
df2_elbow = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_original_elbow = pd.DataFrame(df2_elbow)
df_original_elbow

,k,inertia
0,1,287.000000
1,2,212.123342
2,3,165.136752
3,4,79.022435
4,5,66.413051
5,6,61.634088
6,7,46.074467
7,8,43.392694
8,9,32.913223
9,10,30.469111


In [13]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
df_original_elbow.hvplot.line(
    x="k",
    y="inertia",
    title= "Crypto Elbow Curve",
    xticks=k
)

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [17]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

In [18]:
# Fit the K-Means model using the scaled DataFrame
model.fit(df2)

KMeans(n_clusters=4, random_state=1)

In [19]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
k4 = model.predict(df2)

# Print the resulting array of cluster values.
k4

array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 1, 0, 0, 0, 0])

In [20]:
# Create a copy of the scaled DataFrame
pred = df2.copy()

In [21]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
pred['predicted_cluster'] = k4

# Display the copy of the scaled DataFrame
pred.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,predicted_cluster
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2


In [22]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
pred.hvplot.scatter(
    x='price_change_percentage_24h',
    y='price_change_percentage_7d',
    by='predicted_cluster',
    hover_cols = 'coin_id'
)

:NdOverlay   [predicted_cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [25]:
# Create a PCA model instance and set `n_components=3`.
crypto_pca = PCA(n_components=3)

In [26]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
cluster_pca = crypto_pca.fit_transform(pred)

# View the scaled PCA data
cluster_pca

array([[ 4.48907952e-01, -1.24537573e+00, -8.50640396e-01],
       [ 4.95367093e-01, -8.99822995e-01, -1.31755943e+00],
       [-8.18845712e-01,  7.18990907e-02,  6.95015071e-01],
       [-8.40357106e-01,  8.00540676e-02,  5.44359519e-01],
       [ 8.13240044e-01, -2.66952035e+00, -1.64332113e+00],
       [ 8.22870558e-01, -1.68209607e+00, -9.05203458e-01],
       [ 9.13867626e-01, -1.12868393e+00, -2.73003754e+00],
       [ 8.11102222e-01, -1.11337793e+00, -1.59865515e+00],
       [ 2.58728083e-01, -9.46883804e-01, -1.10355990e+00],
       [-1.00392082e+00, -1.19410202e-01,  2.78563993e-01],
       [-1.36328853e+00,  1.24534877e+00,  9.50712158e-01],
       [-8.30643324e-01,  7.70471599e-02,  7.07660520e-01],
       [-1.13545778e+00,  1.71400623e-01,  4.41569215e-01],
       [ 2.08497797e+00, -2.68897925e+00, -7.76469068e-01],
       [-6.33198072e-01, -1.69113551e-02,  2.49426354e-01],
       [-1.61907992e-01, -3.10904366e-01, -1.81852107e+00],
       [-9.96986646e-01,  1.29257396e+00

In [27]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
crypto_pca.explained_variance_ratio_

array([0.34871677, 0.31363391, 0.22627118])

In [28]:
sum(crypto_pca.explained_variance_ratio_)

0.8886218549859445

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** Sum = 0.8886218549859445

In [74]:
# Scale
scaler = StandardScaler()
df_pca_scaled = scaler.fit_transform(cluster_pca)

# Create a new DataFrame with the PCA data.
df_pca = pd.DataFrame(
    df_pca_scaled,
    columns = ["PCA1", "PCA2", "PCA3"],
    index=df_pca.index
)

# Copy the crypto names from the original scaled DataFrame
df_pca['coin_id'] = list(pred.index)

# Set the coin_id column as index
df_pca = df_pca.set_index('coin_id')

# Display the scaled PCA DataFrame
df_pca

,PCA1,PCA2,PCA3
coin_id,,,
bitcoin,0.269158,-0.787362,-0.633165
ethereum,0.297014,-0.568894,-0.980711
tether,-0.490966,0.045457,0.517327
ripple,-0.503864,0.050612,0.405188
bitcoin-cash,0.487605,-1.687747,-1.223189
binancecoin,0.493379,-1.063469,-0.673779
chainlink,0.547939,-0.713586,-2.032075
cardano,0.486323,-0.703909,-1.189942
litecoin,0.155129,-0.598647,-0.821423


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [78]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,12))

In [80]:
# Create an empty list to store the inertia values
pca_inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    # 2. Fit the model to the data using `df_market_data_scaled`
    k_model.fit(df_pca)
    # 3. Append the model.inertia_ to the inertia list
    pca_inertia.append(k_model.inertia_)


In [81]:
# Create a dictionary with the data to plot the Elbow curve
pca_elbow = {"k": k, "inertia": pca_inertia}
# Create a DataFrame with the data to plot the Elbow curve
df_pca_elbow = pd.DataFrame(pca_elbow)
df_pca_elbow

,k,inertia
0,1,123.000000
1,2,87.912390
2,3,53.683400
3,4,19.123686
4,5,14.381741
5,6,10.476091
6,7,9.015015
7,8,6.771248
8,9,5.900931
9,10,4.240925


In [84]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
df_pca_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="PCA Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No, this is the same value from the original data k value

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [88]:
# Initialize the K-Means model using the best value for k
pca_model = KMeans(n_clusters=4, random_state=1)

In [90]:
# Fit the K-Means model using the PCA data
pca_model.fit(df_pca)

KMeans(n_clusters=4, random_state=1)

In [92]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
k4_pca = pca_model.predict(df_pca)
# Print the resulting array of cluster values.
k4_pca

array([1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 1, 0, 0, 2, 0, 0, 0, 0])

In [94]:
# Create a copy of the scaled PCA DataFrame
df2_pca = df_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
df2_pca['predicted_cluster'] = k4_pca

# Display sample data
df2_pca

,PCA1,PCA2,PCA3,predicted_cluster
coin_id,,,,
bitcoin,0.269158,-0.787362,-0.633165,1
ethereum,0.297014,-0.568894,-0.980711,1
tether,-0.490966,0.045457,0.517327,0
ripple,-0.503864,0.050612,0.405188,0
bitcoin-cash,0.487605,-1.687747,-1.223189,1
binancecoin,0.493379,-1.063469,-0.673779,1
chainlink,0.547939,-0.713586,-2.032075,1
cardano,0.486323,-0.703909,-1.189942,1
litecoin,0.155129,-0.598647,-0.821423,1


In [101]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
df2_pca.reset_index().hvplot.scatter(
    x="PCA1",
    y="PCA2",
    by = 'predicted_cluster',
    hover_cols='coin_id'
)

:NdOverlay   [predicted_cluster]
   :Scatter   [PCA1]   (PCA2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [103]:
# Composite plot to contrast the Elbow curves
# Original Scaled Plot
elbow_original = df_original_elbow.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve Comparison",
    label="Original Data",
    xticks=k,
    line_width=2,
    color="blue"
)

# PCA Scaled Plot
elbow_pca = df_pca_elbow.hvplot.line(
    x="k",
    y="inertia",
    label="PCA Data",
    line_width=2,
    color="green"
)

# Overlay both plots
(elbow_original * elbow_pca).opts(
    width=800,
    height=500,
    xlabel="Number of Clusters (k)",
    ylabel="Inertia",
    legend_position="top_left"
)

:Overlay
   .Curve.Original_Data :Curve   [k]   (inertia)
   .Curve.PCA_Data      :Curve   [k]   (inertia)

In [107]:
# Composite plot to contrast the clusters
df_original_clusters = pred.reset_index()
df_pca_clusters = df2_pca.reset_index()

# Original Clusters
scatter_original = df_original_clusters.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="predicted_cluster",
    hover_cols=["coin_id"],
    alpha=0.6,
    size=100,
    legend=True,
    label="Original Clusters"
).opts(
    width=500, height=500, xlabel="24h Price Change (%)", ylabel="7d Price Change (%)"
)

# PCA Clusters
scatter_pca = df_pca_clusters.hvplot.scatter(
    x="PCA1",
    y="PCA2",
    by="predicted_cluster",
    hover_cols=["coin_id"],
    alpha=0.6,
    size=100,
    legend=True,
    label="PCA Clusters",
    colorbar=True
).opts(
    width=500, height=500, xlabel="PCA1", ylabel="PCA2"
)

# Composite
scatter_original + scatter_pca

:Layout
   .NdOverlay.Original_Clusters :NdOverlay   [predicted_cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.PCA_Clusters      :NdOverlay   [predicted_cluster]
      :Scatter   [PCA1]   (PCA2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** The fewer features used in PCA improves cluster distinction, removes noise, and reduces overfitting to minor variations, which makes it easier to visual isolate groups; however, this comes at the expense of granularity and complexity in the price changes of each coin. While PCA makes it easier to interpret the results via more compact clusters, some important differences in the various coin pricing may get lost in this view. Given the overall variability between the plots is generally minimal, sacrificing details via PCA may not be the best choice in this analysis, particularly when considering the dataset is comprised of pricing information for higher-risk financial assets. Instead, retaining all features perserves greater detail and insights for the purposes of pricing and trend analysis in financial decision making. 